<a href="https://colab.research.google.com/github/mamorim00/lexical_classification/blob/main/Train_Wordnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install

In [ ]:
!pip install transformers
!pip install tensorflow-text
!pip install tf-models-official
!pip install scikit-learn
!pip install keras
!pip install keras_nlp
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 23.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=828c460bfa36f61a946d5a90a25455b9cad66597876874e9456c1249d343fe12
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
import keras_nlp
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer, TFBertModel
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Definindo o início do caminho
caminho = '/content/drive/MyDrive/wordnet_dataset/'

In [ ]:

# Construindo caminhos completos para os arquivos
X_train = pd.read_csv(caminho + 'fold_4_train.csv')
y_train = pd.read_csv(caminho + 'fold_4_train_labels.csv')
X_test = pd.read_csv(caminho + 'fold_4_test.csv')
y_test = pd.read_csv(caminho + 'fold_4_test_labels.csv')


In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np
import datasets
from datasets import Dataset, DatasetDict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# X_train = X_train.head(1000)
# y_train = y_train.head(1000)
# X_test = X_test.head(1000)
# y_test = y_test.head(1000)

In [ ]:
y_test.value_counts()

label
1        17818
0         8701
dtype: int64

In [ ]:
# X_train['def'] = X_train['Definição_Synset'] + "[SEP]" + X_train['Definição_Relacionada']
# X_test['def'] = X_test['Definição_Synset'] + "[SEP]" + X_test['Definição_Relacionada']

In [ ]:
X_train.head()

,def
0,an entity that has physical existence[SEP]that...
1,a general concept formed by extracting common ...
2,a separate and self-contained entity[SEP]an en...
3,a tangible and visible entity; an entity that ...
4,an assemblage of parts that is regarded as a s...


In [ ]:
X_train['label'] = y_train
X_test['label'] = y_test

In [ ]:
# X_train = X_train.drop(['Definição_Synset', 'Definição_Relacionada'], axis=1)
# X_test = X_test.drop(['Definição_Synset', 'Definição_Relacionada'], axis=1)

In [ ]:
dataset_train = Dataset.from_pandas(X_train)
dataset_test = Dataset.from_pandas(X_test)

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [ ]:

def tokenize_function(example):
    return tokenizer(example["def"] ,truncation=True, padding = True, max_length = 256)


tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_test = dataset_test.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_train.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_test.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)



Map:   0%|          | 0/106076 [00:00<?, ? examples/s]

Map:   0%|          | 0/26519 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# Assume tfhub_handle_preprocess and tfhub_handle_encoder are defined somewhere in your code

# Convert DataFrames to NumPy arrays
y_train = y_train.values.ravel()  # Convert to a 1D array
y_test = y_test.values.ravel()    # Convert to a 1D array


In [ ]:



# Calculate class weights based on the training data
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {0: class_weights[0], 1: class_weights[1]}



In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer

# Load BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertForSequenceClassification.from_pretrained(model_name)

# Function to tokenize and prepare input data
def tokenize_batch(batch):
    return tokenizer(batch["input_ids"], batch["attention_mask"], batch["token_type_ids"], padding=True, truncation=True)

# Create a simple model using BERT
input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
token_type_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="token_type_ids")

bert_output = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)[0]

# Add dropout layer
dropout_rate = 0.1  # You can adjust this value based on your needs
dropout_layer = tf.keras.layers.Dropout(rate=dropout_rate)(bert_output)


# Add additional layers as needed
# For example, you can add a Dense layer with sigmoid activation for binary classification
dense_layer = tf.keras.layers.Dense(1, activation='sigmoid')(bert_output)

# Build the model
model = tf.keras.Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=dense_layer)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras import metrics

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
)


In [ ]:
# Print model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 token_type_ids (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                            

In [ ]:
model.fit(tf_train_dataset, epochs=2, batch_size= 32, class_weight= class_weights)

Epoch 1/2
13260/13260 [==============================] - 1506s 110ms/step - loss: 0.1569 - accuracy: 0.9409 - precision_1: 0.9670 - recall_1: 0.9442
Epoch 2/2
13260/13260 [==============================] - 1455s 110ms/step - loss: 0.0647 - accuracy: 0.9778 - precision_1: 0.9893 - recall_1: 0.9775


In [ ]:
# model.evaluate(tf_test_dataset)

In [ ]:
# Initialize a dictionary to store metrics for different folds
metrics_dict = {}

# Specify the fold number
fold_number = 4

# # Evaluate the model on the test dataset
evaluation_result = model.evaluate(tf_test_dataset)

# Add the evaluation result to the dictionary with the fold number as the key
metrics_dict[fold_number] = evaluation_result

# Now metrics_dict contains the evaluation metrics for fold 1
print("Metrics Dictionary:", metrics_dict)

3315/3315 [==============================] - 150s 44ms/step - loss: 0.0810 - accuracy: 0.9736 - precision_1: 0.9807 - recall_1: 0.9800
Metrics Dictionary: {4: [0.08096833527088165, 0.9735661149024963, 0.9806795716285706, 0.9799640774726868]}


In [ ]:
model.predict(tf_test_dataset)

3890/3890 [==============================] - 166s 42ms/step


array([[0.00117313],
       [0.8636039 ],
       [0.9914131 ],
       ...,
       [0.99081063],
       [0.99835557],
       [0.00117481]], dtype=float32)

In [ ]:
Metrics_Dictionary: {1: [0.07133997976779938, 0.9759795069694519, 0.9819895625114441, 0.9822651147842407]}

In [ ]:
{3: [0.08647516369819641, 0.9735283851623535, 0.9886934757232666, 0.9717140197753906]}

In [ ]:
Metrics_Dictionary = {1: [0.07133997976779938, 0.9759795069694519, 0.9819895625114441, 0.9822651147842407], 2: [0.07019591331481934, 0.974848210811615, 0.9834808707237244, 0.9790099859237671],3: [0.08647516369819641, 0.9735283851623535, 0.9886934757232666, 0.9717140197753906],4: [0.08096833527088165, 0.9735661149024963, 0.9806795716285706, 0.9799640774726868],5: [0.06670548766851425, 0.9769976139068604, 0.9845131635665894, 0.981197714805603]}

In [ ]:
Metrics_Dictionary

{1: [0.07133997976779938,
  0.9759795069694519,
  0.9819895625114441,
  0.9822651147842407],
 2: [0.07019591331481934,
  0.974848210811615,
  0.9834808707237244,
  0.9790099859237671],
 3: [0.08647516369819641,
  0.9735283851623535,
  0.9886934757232666,
  0.9717140197753906],
 4: [0.08096833527088165,
  0.9735661149024963,
  0.9806795716285706,
  0.9799640774726868],
 5: [0.06670548766851425,
  0.9769976139068604,
  0.9845131635665894,
  0.981197714805603]}

In [ ]:
import numpy as np

# Function to calculate F1 score
def calculate_f1(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

# Your existing Metrics_Dictionary
Metrics_Dictionary = {
    1: [0.07133997976779938, 0.9759795069694519, 0.9819895625114441, 0.9822651147842407],
    2: [0.07019591331481934, 0.974848210811615, 0.9834808707237244, 0.9790099859237671],
    3: [0.08647516369819641, 0.9735283851623535, 0.9886934757232666, 0.9717140197753906],
    4: [0.08096833527088165, 0.9735661149024963, 0.9806795716285706, 0.9799640774726868],
    5: [0.06670548766851425, 0.9769976139068604, 0.9845131635665894, 0.981197714805603]
}

# Update Metrics_Dictionary with F1 scores
for fold, metrics in Metrics_Dictionary.items():
    precision = metrics[2]  # Index 2 is precision
    recall = metrics[3]     # Index 3 is recall
    f1_score = calculate_f1(precision, recall)
    metrics.append(f1_score)

# Print updated Metrics_Dictionary
for fold, metrics in Metrics_Dictionary.items():
    print(f"Fold {fold}: Loss={metrics[0]}, Accuracy={metrics[1]}, Precision={metrics[2]}, Recall={metrics[3]}, F1 Score={metrics[4]}")


Fold 1: Loss=0.07133997976779938, Accuracy=0.9759795069694519, Precision=0.9819895625114441, Recall=0.9822651147842407, F1 Score=0.9821273193201412
Fold 2: Loss=0.07019591331481934, Accuracy=0.974848210811615, Precision=0.9834808707237244, Recall=0.9790099859237671, F1 Score=0.9812403356093449
Fold 3: Loss=0.08647516369819641, Accuracy=0.9735283851623535, Precision=0.9886934757232666, Recall=0.9717140197753906, F1 Score=0.9801302166276236
Fold 4: Loss=0.08096833527088165, Accuracy=0.9735661149024963, Precision=0.9806795716285706, Recall=0.9799640774726868, F1 Score=0.9803216939986318
Fold 5: Loss=0.06670548766851425, Accuracy=0.9769976139068604, Precision=0.9845131635665894, Recall=0.981197714805603, F1 Score=0.9828526432000212


In [ ]:
# Calculate average metrics
avg_metrics = np.mean(list(Metrics_Dictionary.values()), axis=0)

# Print average metrics
print(f"Average Metrics: Loss={avg_metrics[0]}, Accuracy={avg_metrics[1]}, Precision={avg_metrics[2]}, Recall={avg_metrics[3]}, F1 Score={avg_metrics[4]}")


Average Metrics: Loss=0.0751369759440422, Accuracy=0.9749839663505554, Precision=0.983871328830719, Recall=0.9788301825523377, F1 Score=0.9813344417511527


In [ ]:
model_saved = model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
